In [ ]:
cd ..

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")

import json
import numpy as np
import matplotlib.pyplot as plt

import cv2
import tensorflow as tf

from keras.applications import EfficientNetB0
from keras.models import Model, Sequential
from keras.layers import Input, InputLayer, GlobalAveragePooling2D, Dropout, Dense

from sklearn.model_selection import KFold

# Pith Dataset

In [ ]:
def get_image_path(directory, image_name):
    return os.path.join(directory, image_name)

In [ ]:
with open(r'data/Log_Ends_Train.json', 'r') as f:
    PITH_JSON_TRAIN = json.load(f)
    
with open(r'data/Log_Ends_Val.json', 'r') as f:
    PITH_JSON_VAL = json.load(f)

In [ ]:
def get_bbox(image_name, json_file):
    for image_item in json_file["images"]:
        if image_item["image_name"] == image_name:
            return image_item["labels"][0]["bbox"]
    return None

def get_xy_coordinates(image, image_name, json_file):
    x,y,w,h = get_bbox(image_name, json_file)
    w = np.abs(w-x)
    h = np.abs(h-y)
    height, width = image.shape[0], image.shape[1]
    return (x + w/2)/width, (y + h/2)/height

In [ ]:
#STOP

In [ ]:
labels_Log_Ends_Train = []
images_Log_Ends_Train = []
for image_item in PITH_JSON_TRAIN["images"]:
    image_name = image_item["image_name"]
    image_path = get_image_path(r'data/Log_Ends_Train', image_name)
    image = plt.imread(image_path)
    labels_Log_Ends_Train.append(get_xy_coordinates(image, image_name, PITH_JSON_TRAIN)) 
    images_Log_Ends_Train.append(image)

labels_Log_Ends_Val = []
images_Log_Ends_Val = []
for image_item in PITH_JSON_VAL["images"]: 
    image_name = image_item["image_name"]
    image_path = get_image_path(r'data/Log_Ends_Val', image_name)
    image = plt.imread(image_path)
    labels_Log_Ends_Val.append(get_xy_coordinates(image, image_name, PITH_JSON_VAL))
    images_Log_Ends_Val.append(image)

In [ ]:
prediction = labels_Log_Ends_Train[0]
img = images_Log_Ends_Train[0]
height, width = img.shape[0], img.shape[1]
plt.imshow(img)
plt.plot(prediction[0]*width, prediction[1]*height, '.', markersize = 10)
plt.show()

In [ ]:
prediction =labels_Log_Ends_Val[0]
img = images_Log_Ends_Val[0]
height, width = img.shape[0], img.shape[1]
plt.imshow(img)
plt.plot(prediction[0]*width, prediction[1]*height, '.', markersize = 10)
plt.show()

In [ ]:
#STOP

# Config

In [ ]:
epochs, batch_size = 50, 32
kf = KFold(n_splits=3, shuffle=True, random_state=42)

early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True,
    )

METRICS = [ 
    tf.keras.metrics.MAE,
    tf.keras.metrics.MSE,
    tf.keras.metrics.RootMeanSquaredError()
]

IMG_SIZE = 224 # Image resolution

In [ ]:
#STOP

# Model

In [ ]:
class Baseline():
    def __init__(self, metrics=METRICS):
        # Define the input shape
        '''
        input_shape = (224, 224, 3)
        
        # Instantiate the base model (EfficientNetB0)
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

        # Freeze the weights of the base model
        #base_model.trainable = False

        # Create a new model on top of the base model
        model = Sequential([
            InputLayer(input_shape=input_shape),
            base_model,
            GlobalAveragePooling2D(),
            Dropout(0.5),
            Dense(2, activation='softmax')
        ])
        '''
        
        #
        input_shape = (224, 224, 3)
        inputs = Input(shape=input_shape)
        x = EfficientNetB0(include_top=False, input_shape=input_shape)(inputs)
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.5)(x)
        prediction = Dense(2, activation='softmax')(x)
        model = Model(inputs=inputs, outputs=prediction)

        # Compile the model
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), metrics=metrics)

        self.model = model

    def train(self, X_train, y_train, X_val, y_val, epochs, batch_size):
        #history = self.model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=2)
        history = self.model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, verbose=2)
        return history
    
    def evaluate(self, X_test, y_test):
        results = self.model.evaluate(X_test, y_test, verbose=0)
        return results

    def predict(self, X_test):
        predictions = self.model.predict(X_test)
        return predictions

    def summary(self):
        self.model.summary()
        
    def metrics_names(self):
        return self.model.metrics_names

# Prepare dataset

In [ ]:
resized_images_Log_Ends_Train = [cv2.resize(img, (IMG_SIZE, IMG_SIZE)) for img in images_Log_Ends_Train]
resized_images_Log_Ends_Val = [cv2.resize(img, (IMG_SIZE, IMG_SIZE)) for img in images_Log_Ends_Val]

X_train = np.array(resized_images_Log_Ends_Train)
y_train = np.array(labels_Log_Ends_Train)

X_holdout = np.array(resized_images_Log_Ends_Val)
y_holdout = np.array(labels_Log_Ends_Val)

In [ ]:
# create the model and iterate over each fold

model = Baseline()
model.summary()

In [ ]:
#STOP

In [ ]:
results_loss = []
results_mae = []
results_mse = []
results_rmse = []

for train_index, val_index in kf.split(X_train):
    # split the dataset into training and validation sets for this fold
    X_train_kf, X_val_kf = X_train[train_index], X_train[val_index]
    y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]
    
    history = model.train(X_train_kf, y_train_kf, X_val_kf, y_val_kf, epochs, batch_size)
    plt.plot(history.history['loss'])
    plt.show()
    
    loss, mae, mse, rmse = model.evaluate(X_val_kf, y_val_kf)
    print('Loss on validation set:', loss)
    print('MAE on validation set:', mae)
    print('MSE on validation set:', mse)
    print('RMSE on validation set:', rmse)

    results_loss.append(loss)
    results_mae.append(mae)
    results_mse.append(mse)
    results_rmse.append(rmse)
    
# Calculate the average for the K-fold cross-validation
print(f"K-fold cross-validation Loss: {sum(results_loss) / len(results_loss)}")
print(f"K-fold cross-validation MAE: {sum(results_mae) / len(results_mae)}")
print(f"K-fold cross-validation MSE: {sum(results_mse) / len(results_mse)}")
print(f"K-fold cross-validation RMSE: {sum(results_rmse) / len(results_rmse)}")

loss, mae, mse, rmse = model.evaluate(X_holdout, y_holdout)
print('Loss on holdout set:', loss)
print('MAE on holdout set:', mae)
print('MSE on holdout set:', mse)
print('RMSE on holdout set:', rmse)

In [ ]:
#STOP

In [ ]:
img_ind = 1

image = images_Log_Ends_Train[img_ind]
height, width = image.shape[0], image.shape[1]
image = cv2.resize(image, (224,224))
image = np.expand_dims(image, axis=0)
prediction = model.predict(image)[0]
image = np.squeeze(image, axis = 0)
image = cv2.resize(image, (width, height))
plt.imshow(image)
plt.plot(prediction[0]*width, prediction[1]*height, '.', markersize = 10)
plt.title("Predicted")
plt.show()

prediction = labels_Log_Ends_Train[img_ind]
img = images_Log_Ends_Train[img_ind]
height, width = img.shape[0], img.shape[1]
plt.imshow(img)
plt.plot(prediction[0]*width, prediction[1]*height, '.', markersize = 10)
plt.title("Original")
plt.show()

In [ ]:
img_ind = 1


image = images_Log_Ends_Val[1]
height, width = image.shape[0], image.shape[1]
image = cv2.resize(image, (224,224))
image = np.expand_dims(image, axis=0)
prediction = model.predict(image)[0]
image = np.squeeze(image, axis = 0)
image = cv2.resize(image, (width, height))
plt.imshow(image)
plt.plot(prediction[0]*width, prediction[1]*height, '.', markersize = 10)
plt.title("Predicted")
plt.show()

prediction = labels_Log_Ends_Val[img_ind]
img = images_Log_Ends_Val[img_ind]
height, width = img.shape[0], img.shape[1]
plt.imshow(img)
plt.plot(prediction[0]*width, prediction[1]*height, '.', markersize = 10)
plt.title("Original")
plt.show()

In [ ]:
#STOP